In [44]:
"""
This code is for constructing labeled preference pairs.
"""
import os
import json
from tqdm import tqdm
import numpy as np
import random
from random import shuffle
random.seed(42)

## 1. UltraFeedback (AI Feedback)

In [2]:
ultra_feedback_dir = "UltraFeedback"

In [3]:
def read_ultra_feedback(data_dir):
    all_files = os.listdir(data_dir)
    print(all_files)
    examples = list()
    for data_file in tqdm(all_files):
        if ".jsonl" not in data_file:
            continue
        with open(os.path.join(data_dir, data_file), "r", encoding="utf-8") as fr:
            for line in fr.readlines():
                examples.append(json.loads(line))
    return examples
ultra_feedback_examples = read_ultra_feedback(ultra_feedback_dir)

['ultrachat.jsonl', 'truthful_qa.jsonl', '.gitattributes', 'flan.jsonl', 'false_qa.jsonl', 'sharegpt.jsonl', '.huggingface', 'README.md', 'evol_instruct.jsonl']


100%|██████████| 9/9 [00:08<00:00,  1.09it/s]


In [41]:
def obtain_pair_from_ultra_feedback(examples):
    """
    Construct pair-wise data for preference optimzation
    return :the structure of example:
    
      {
        "conversations": [
          {
            "from": "system",
            "value": "You are a teacher. Given a task, you explain in simple steps what the task is asking, any guidelines it provides and how to use those guidelines to find the answer."
          },
          {
            "from": "human",
            "value": "Q: In this task, you need to provide the parts-of-speech tag of a word present in a sentence specified within curly braces  ( '{{ ... }}' ). The parts-of-speech tags are coarse labels that represent a category of words with similar grammatical properties. The list of part-of-speech tags i.e tagset of this corpus is - \n '.': Period symbol is used for symbols denoting Punctuations/Separations such as comma, period, backticks etc., \n 'ADJ': Adjectives are words that typically modify nouns and specify their properties or attributes, \n 'ADP': Adposition is a cover term for prepositions and postpositions, \n 'ADV': Adverbs are words that typically modify verbs for such categories as time, place, direction or manner, \n 'CONJ': A word used to connect clauses or sentences or to coordinate words in the same clause, \n 'DET': Determiners are words that modify nouns or noun phrases and express the reference of the noun phrase in context, \n 'NOUN': Nouns are a part of speech typically denoting a person, place, thing, animal or idea, \n 'NUM': A numeral is a word, functioning most typically as a determiner, adjective or pronoun, that expresses a number and a relation to the number, such as quantity, sequence, frequency or fraction, \n 'PRT': Particles are function words that must be associated with another word or phrase to impart meaning and that do not satisfy definitions of other universal parts of speech, \n 'PRON': Pronouns are words that substitute for nouns or noun phrases, whose meaning is recoverable from the linguistic or extralinguistic context, \n 'PROPN': A proper noun is a noun (or nominal content word) that is the name (or part of the name) of a specific individual, place, or object, \n 'VERB': A verb is a member of the syntactic class of words that typically signal events and actions, can constitute a minimal predicate in a clause, and govern the number and types of other constituents which may occur in the clause, \n 'X': The tag X is used for words that for some reason cannot be assigned a real part-of-speech category.\nSentence: `` I'm not worried about it '' , Phil said {{ . }} \nWord: .\nA:"
          }
        ],
        "chosen": {
          "from": "assistant",
          "value": "In this task, you are asked to identify the part-of-speech (POS) tag for a word in a given sentence. The word is specified within curly braces ({{...}}). The corpus follows a predefined tagset that indicates different parts-of-speech categories.\n\nTo complete this task:\n\n1. Carefully read the list of part-of-speech tags provided.\n2. Analyze the sentence given, paying attention to the word within the curly braces: \"{{ . }}\"\n3. Look through the tagset and determine which tag best fits the word within the curly braces.\n4. Provide the appropriate part-of-speech tag for the word.\n\nFor the given sentence: \"I'm not worried about it\", Phil said {{ . }}\nWord: .\n\nUsing the guidelines, we can see that the word '.' is a punctuation symbol. According to the tagset:\n'.': Period symbol is used for symbols denoting Punctuations/Separations such as comma, period, backticks, etc.\n\nSo, the part-of-speech tag for the word '.' is: '.'"
        },
        "rejected": {
          "from": "assistant",
          "value": "Sure, I'd be happy to help! Here are the steps to find the part of speech of the word in the given sentence:\n\nStep 1: Read the sentence and identify the word you want to find the part of speech for. In this case, the word is \"it\".\n\nStep 2: Look at the list of part-of-speech tags provided in the task. In this case, we have the following tags:\n\n'.': Period symbol (punctuation)\nADJ: Adjective\nADP: Adposition (preposition or postposition)\nADV: Adverb\nCONJ: Conjunction\nDET: Determiner\nNOUN: Noun\nNUM: Numeral\nPRT: Particle\nPRON: Pronoun\nPROPN: Proper noun\nVERB: Verb\nX: Unclassified word (for words that cannot be assigned a real part-of-speech category)\n\nStep 3: Look at the word \"it\" in the sentence and determine which category it best fits into. In this case, \"it\" is a pronoun, so the part of speech is PRON.\n\nTherefore, the part of speech of the word \"it\" in the sentence \"I'm not worried about it\", Phil said is PRON."
        }
      }
    """
    preference_pair_examples = list()
    for example in tqdm(examples):
        instruction = example["instruction"]
        completions = example["completions"]
        model_list = example["models"]
        score_list = list()
        system_prompt_list = list()
        response_list = list()
        for response_result in completions:
            custom_system_prompt = response_result["custom_system_prompt"]
            fine_score = response_result["fine-grained_score"]
            overall_score = response_result["overall_score"]
            response = response_result["response"]
            
            score_list.append(fine_score + overall_score)
            system_prompt_list.append(custom_system_prompt)
            response_list.append(response)
        
        if len(score_list) < 2:
            continue
        # select the highest score
        accepted_response_idx = np.argmax(score_list)
        rejected_response_idx = -1
        # random select one lower score
        num = 5
        while num >= 0:
            num -= 1
            rejected_response_idx = random.choice(list(range(len(model_list))))
            if rejected_response_idx != accepted_response_idx:
                break
        if rejected_response_idx == accepted_response_idx:
            continue
        preference_pair_examples.append({
            "conversations": [
                {
                    "from": "system",
                    "value": system_prompt_list[accepted_response_idx],
                },
                {
                    "from": "human",
                    "value": instruction,
                }
            ],
            "chosen": {
                # "from": model_list[accepted_response_idx],
                "from": "assistant",
                "value": response_list[accepted_response_idx]
            },
            "rejected": {
                # "from": model_list[rejected_response_idx],
                "from": "assistant",
                "value": response_list[rejected_response_idx]
            }
        })
    return preference_pair_examples
        
ultra_feedback_preference_pair_examples = obtain_pair_from_ultra_feedback(ultra_feedback_examples)

100%|██████████| 63967/63967 [00:00<00:00, 65769.99it/s]


In [62]:
def obtain_cls_from_preference_pair_ultra_feedback(examples):
    """
    Construct cls style for preference optimzation
    input: ultra_feedback_preference_pair_examples
    return :the structure of example:
    
      {
        "instruction": "You are a good preference labeler. Give you one query from human and two candidate responses answered by the same assistant, your job is to find the better response which is more helpful, harmless for the human query.\n\nQ: xxx\n\n\nResponse #1: \n\n\nReponse #2",
        "label": "0"
      }
    """
    ultra_feedback_cls_examples = list()
    for example in tqdm(examples):
        accepted, rejected = example["chosen"]["value"], example["rejected"]["value"]
        query = example["conversations"][1]["value"]
        ultra_feedback_cls_examples.append({
            "instruction": f"You are a good preference labeler. Give you one query from human and two candidate responses answered by the same assistant, your job is to find the better response which is more helpful, harmless for the human query.\n\nQ: {query}\n\n\nResponse #1: {accepted} \n\n\nReponse #2: {rejected}",
            "label": 0
        })
        ultra_feedback_cls_examples.append({
            "instruction": f"You are a good preference labeler. Give you one query from human and two candidate responses answered by the same assistant, your job is to find the better response which is more helpful, harmless for the human query.\n\nQ: {query}\n\n\nResponse #1: {rejected} \n\n\nReponse #2: {accepted}",
            "label": 1
        })
    shuffle(ultra_feedback_cls_examples)
    return ultra_feedback_cls_examples
ultra_feedback_cls_examples = obtain_cls_from_preference_pair_ultra_feedback(ultra_feedback_preference_pair_examples)


100%|██████████| 63953/63953 [00:00<00:00, 163022.58it/s]


In [63]:
with open("./UltraFeedback/pair_train_data.json", "w", encoding="utf-8") as fw:
    for example in tqdm(ultra_feedback_preference_pair_examples):
        fw.write(json.dumps(example, ensure_ascii=False) + "\n")

with open("./UltraFeedback/cls_train_data.json", "w", encoding="utf-8") as fw:
    for example in tqdm(ultra_feedback_cls_examples):
        fw.write(json.dumps(example, ensure_ascii=False) + "\n")

100%|██████████| 127906/127906 [00:10<00:00, 12565.54it/s]


In [30]:
len(ultra_feedback_preference_pair_examples)

63952

In [40]:
ultra_feedback_preference_pair_examples[50030]

{'conversations': [{'from': 'system',
   'value': 'As an AI assistant, conduct yourself with honesty. Always be diligent in giving accurate responses. Fictional facts or inaccuracies should be avoided in your responses.'},
  {'from': 'human',
   'value': 'Can you make me a code to illustrate a molecule using its SMILES?'}],
 'chosen': {'from': 'assistant',
  'value': "I'd be happy to help! However, I must point out that the question contains some ambiguity. SMILES (Simplified Molecular Input Line Entry System) is a notation for describing the structure of molecules using short strings of characters. It is not a programming language, and it cannot be used to create executable code.\n\nIf you meant to ask for a code that can generate a visual representation of a molecule from its SMILES string, I can provide you with some suggestions. There are several libraries and tools available that can help with this task, such as RDKit, PyMOL, or Matplotlib.\n\nHere's an example of how you could us

In [54]:
idx = 10001
print(ultra_feedback_cls_examples[idx]["prompt"])
print(ultra_feedback_cls_examples[idx]["label"])

You are a good preference labeler. Give you one query from human and two candidate responses answered by the same assistant, your job is to find the better response which is more helpful, harmless for the human query.

Q: In your research on the impact of guns on domestic violence, provide statistics on the frequency and severity of gun-related domestic violence incidents, examine the laws and policies that regulate firearm ownership and use in cases of domestic violence, discuss the psychological effects of gun presence on domestic violence victims, and propose potential solutions (such as stricter gun control or enhanced victim support services) to mitigate the impact of guns on domestic violence. Your presentation should include both factual information and personal anecdotal accounts to illustrate the gravity and complexity of this issue.


Response #1: Title: The Impact of Guns on Domestic Violence

Slide 1: Introduction

* Definition of domestic violence
* Overview of the topic: 